In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [2]:
f = Foo()

In [3]:
f[1]

10

In [4]:
for i in f:print(i)

0
10
20


In [5]:
20 in f

True

In [7]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [8]:
from random import shuffle

In [9]:
l = list(range(10))

In [11]:
shuffle(l)

In [12]:
l

[1, 4, 9, 7, 2, 6, 3, 5, 8, 0]

In [13]:
deck = FrenchDeck()

In [14]:
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [15]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [16]:
FrenchDeck.__setitem__ = set_card

In [17]:
shuffle(deck)

In [19]:
deck[:5]

[Card(rank='7', suit='diamonds'),
 Card(rank='J', suit='hearts'),
 Card(rank='9', suit='hearts'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='clubs')]

In [20]:
shuffle(deck)

In [21]:
deck[:5]

[Card(rank='9', suit='clubs'),
 Card(rank='10', suit='spades'),
 Card(rank='10', suit='hearts'),
 Card(rank='A', suit='diamonds'),
 Card(rank='9', suit='hearts')]

In [28]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc
isinstance(Struggle(), abc.Sized)

In [31]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)

In [1]:
import abc

class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""
    
    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回
        如果实例为空，这个方法应该抛出 LookupError
        """
    
    def loaded(self):
        """如果至少有一个元素，返回True，否则返回False"""
        return bool(self.inspect())
    
    def inspect(self):
        """返回一个有序元组，由当前元素构成"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [2]:
class Fake(Tombola):
    def pick(self):
        return 13

In [3]:
Fake

__main__.Fake

In [4]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [5]:
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, ...):
        pass

SyntaxError: invalid syntax (<ipython-input-5-6136d0fe469f>, line 4)

In [2]:
import random

In [3]:
class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [4]:
class LotteryBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

In [5]:
from random import randrange

In [14]:
@Tombola.register
class TomboList(list):
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
        
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    
# Tombola.register(TomTomboList)

In [15]:
issubclass(TomboList, Tombola)

True

In [16]:
t = TomboList(range(100))

In [17]:
isinstance(t, Tombola)

True

In [18]:
type(t)

__main__.TomboList

In [19]:
TomboList.__mro__

(__main__.TomboList, list, object)

In [20]:
import doctest

In [21]:
TEST_FILE = 'tombola_tests.rst'
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'

In [22]:
def main(argv):
    verbose = '-v' in argv
    real_subclasses = Tombola.__subclasses__()
    virtual_subclasses = list(Tombola._abc_registry)
    
    for cls in real_subclasses + virtual_subclasses:
        test(cls, verbose)

In [23]:
def test(cls, verbose=False):
    res = doctest.testfile(
        TEST_FILE,
        globs={'ConcreteTombola': cls},
        verbose=verbose,
        optionflags=doctest.REPORT_ONLY_FIRST_FAILURE
    )
    tag = 'FAIL' if res.failed else 'ok'
    print(TEST_MSG.format(cls.__name__, res, tag))

In [24]:
if __name__ == '__main__':
    import sys
    main(sys.argv)

AttributeError: type object 'Tombola' has no attribute '_abc_registry'